In [1]:
!pip install flask_sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=b6e054cf091977cecb9f5bf1799d2b2d80d94d446bedece884f1c6bbc2d3a9d0
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [4]:
!ngrok authtoken '294dzGF6SRGFWn5l0brOtz37HvR_2WRzjVis8V5QuYW9A7Gda'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install DBCreateScript

In [ ]:
from flask import Flask, render_template, request, jsonify, flash
from flask_ngrok import run_with_ngrok
from flask_sqlalchemy import SQLAlchemy
# from DBCreateScript import db_object
from google.colab import drive
import json
from flask import Response
from functools import wraps

drive.mount('/content/gdrive')

import urllib
import os
import sys
sys.path.append('/content/gdrive//MyDrive/VoiceRecognition/main/')
import record_voice
import Predict

TEMPLATE_FOLDER = '/content/gdrive//MyDrive/VoiceRecognition/templates' # 임의의 경로
STATIC_FOLDER = '/content/gdrive//MyDrive/VoiceRecognition/static' # 임의의 경로
app = Flask(__name__, template_folder=TEMPLATE_FOLDER, static_folder = STATIC_FOLDER)
run_with_ngrok(app)
app.config['JSON_AS_ASCII'] = False
app.config["SECRET_KEY"] = "ABCD"

# app.config['SQLALCHEMY_DATABASE_URI'] = db_object.connection()
# app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

#db = SQLAlchemy(app)

def as_json(f):
  @wraps(f)
  def decorated_function(*args, **kwargs):
    res = f(*args, **kwargs)
    res = json.dumps(res, ensure_ascii=False).encode('utf8')
    return Response(res, content_type='application/json; charset=utf-8')
  return decorated_function

#목소리 등록 버튼 클릭 시
@app.route('/register', methods=['POST'])
def register_voice():
  data = request.get_json()
  name = data['name']
  age = data['age']
  sex = data['sex']

  print(name+age+sex)

  record_voice.record(name,age,sex)

  return jsonify(result="success")

#등록된 목소리 확인 버튼 클릭 시
@app.route('/voicecheck/<data>')
def check_voice(data):
  user_info=data
  print(user_info)

  voicelist=[]

  path = "/content/gdrive//MyDrive/VoiceRecognition/static/Recording/"

  if not os.path.isdir(path+user_info):
    flash("등록된 목소리가 없습니다. 목소리 등록을 시도해주십시오.")
    return render_template('Checkvoice.html')
  else:
    for list in os.listdir(path+user_info):
      voicelist.append('/static/Recording/'+user_info+'/'+list)
      print('/Recording/'+user_info+'/'+list)
    print(voicelist)
    return render_template('Checkvoice.html',voicelist=voicelist)

#화자인식 버튼 클릭 시
@app.route('/recognition/<data>')
def regcognition(data):
  user_info=data

  path = "/content/gdrive//MyDrive/VoiceRecognition/static/Recording/"

  if not os.path.isdir(path+user_info):
    flash("등록된 목소리가 없습니다. 목소리 등록을 시도해주십시오.")
    return render_template('Recognition.html')
  else:
    print("화자인식 버튼 클릭 후 화자인식 진행창 띄우기")
    print(user_info)

    return render_template('Recognition.html',user_info=user_info)

#화자인식 페이지의 인식 버튼 클릭 시
@app.route('/recording_recognition', methods=['POST'])
def recording_recognition():
  data = request.get_json() #김민지22female
  user_info=urllib.parse.unquote(data)
  print(user_info)

  recognition_audio=record_voice.recognition_record()

  print("화자인식 녹음 종료")

  #화자인식 메소드
  result=Predict.result_recognition(user_info)

  return jsonify(result=result)

@app.route('/')
def home():
  return render_template('Registration.html')

if __name__ == '__main__':
  app.run()

Mounted at /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/gdrive//MyDrive/VoiceRecognition/main/', '/content/gdrive//MyDrive/VoiceRecognition/main/', '/content/gdrive/MyDrive/VoiceRecognition/main']
Mounted at /content/gdrive
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8990-34-145-193-143.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Jun/2022 20:52:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 20:52:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 20:52:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 20:52:46] "GET /static/js/Registration.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 20:52:46] "GET /static/js/Registration.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 20:52:46] "GET /static/css/css.css HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 20:52:46] "GET /static/css/css.css HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 20:52:46] "GET /static/js/Recognition.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 20:52:46] "GET /static/js/Recognition.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 20:52:47] "GET /favicon.ico HTTP/1.1" 404 -


화자인식 버튼 클릭 후 화자인식 진행창 띄우기
정준호28male


...

<IPython.core.display.Javascript object>

화자인식 녹음 종료
['/content/gdrive//MyDrive/VoiceRecognition/static/Recording/박은주35female', '/content/gdrive//MyDrive/VoiceRecognition/static/Recording/박은주35female', '/content/gdrive//MyDrive/VoiceRecognition/static/Recording/박은주35female', '/content/gdrive//MyDrive/VoiceRecognition/static/Recording/박은주35female', '/content/gdrive//MyDrive/VoiceRecognition/static/Recording/박은주35female', '/content/gdrive//MyDrive/VoiceRecognition/static/Recording/박은주35female', '/content/gdrive//MyDrive/VoiceRecognition/static/Recording/박은주35female', '/content/gdrive//MyDrive/VoiceRecognition/static/Recording/박은주35female', '/content/gdrive//MyDrive/VoiceRecognition/static/Recording/박은주35female', '/content/gdrive//MyDrive/VoiceRecognition/static/Recording/박은주35female', '/content/gdrive//MyDrive/VoiceRecognition/static/Recording/정효인22female', '/content/gdrive//MyDrive/VoiceRecognition/static/Recording/정효인22female', '/content/gdrive//MyDrive/VoiceRecogniti

127.0.0.1 - - [12/Jun/2022 21:00:57] "POST /recording_recognition HTTP/1.1" 200 -


예측한 값 정준호28male
실제 값 손동혁29male
예측한 값 길이9
실제 값 길이9
failed


127.0.0.1 - - [12/Jun/2022 21:07:13] "GET /recognition/정주노29male HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 21:07:14] "GET /recognition/정주노29male HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 21:07:15] "GET /static/css/css.css HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 21:07:15] "GET /static/js/Recognition.js HTTP/1.1" 200 -
